# Boston 房价预测

数据集: 本数据集由StatLib库取得，由CMU维护。每个样本都是1970s晚期波士顿郊区的不同位置，每条数据含有13个属性，目标值是该位置房子的房价中位数（千dollars）。

任务: 预测房价,回归任务.

### 导入模块

In [1]:
from keras.datasets import boston_housing
from keras import models
from keras import layers
import numpy as np

Using TensorFlow backend.


### 导入数据集

In [2]:
# 数据集下载
(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

### 探索数据集

训练集,测试集shape,每条记录内容.

In [3]:
print(train_data.shape, test_data.shape)

(404, 13) (102, 13)


每条数据内容:

In [4]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

每个特征取值范围不同,在送到网络之前,需要进行预处理,将数据特征归一化,缩放到同一取值范围内;如果不做预处理,不同特征对loss函数的贡献不同,梯度优化时会有所偏向.

### 数据预处理

归一化处理, (x-mean)/std; 减去均值,除以标准差.

测试集的均值和方差来自于训练集,不会根据测试集重新计算!

In [5]:
# 数据处理
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

### 模型构建

确定网络架构.

In [6]:
# 模型构建
def build_models():
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    # 编译过程
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    
    return model

### 数据集切分

由于训练集过小,采用k-fold交叉验证.验证方法如下:

![K-fold交叉验证](https://ars.els-cdn.com/content/image/1-s2.0-S0306261917313429-gr2.jpg)

将训练集划分成K分,其中K-1份用来训练,另一份用来测试,最后将K个测试结果取平均作为最终的测试结果.

In [7]:
# 数据集切分,采用4折交叉验证方法.
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #',i)
    # 当前验证集数据
    val_data = train_data[i*num_val_samples : (i+1)*num_val_samples] # 划分出验证集部分
    val_targets = train_targets[i*num_val_samples : (i+1)*num_val_samples]
    # 训练集,将前,后两部分拼在一起,axis=0,down纵向拼接---数据增多,特征没有变化
    partial_train_data = np.concatenate([train_data[:i*num_val_samples],train_data[(i+1)* num_val_samples:] ],axis=0) # 将训练集拼接到一起
    partial_train_targets = np.concatenate([train_targets[:i*num_val_samples],train_targets[(i+1)* num_val_samples:] ],axis=0)

    # 训练
    model = build_models()
    model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,batch_size=16,verbose=0)#模型训练silent模型
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0) # 验证集上评估
    # 保存测试结果
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


测试结果:

In [10]:
all_scores

[1.9035061727655995, 2.431620779604015, 2.5146334171295166, 2.370800970804573]

均值大小为:

In [8]:
np.mean(all_scores)

2.3138774267517697

### 重新训练

使用全部训练集训练,训练完之后,使用测试集验证,查看验证结果.

In [9]:
model = build_models()
model.fit(train_data,train_targets,epochs=80,batch_size=16,verbose=0)
mes,mae = model.evaluate(test_data,test_targets)

102/102 [==============================] - 0s 411us/step


误差大小:

In [10]:
mes

20.8359769933364

mae大小(越小越好):

In [11]:
mae

3.0203938110201967

### 小结

回归模型:loss函数一般为mse,也可以是mae.

回归任务和分类任务处理流程相似.

1. 数据处理;
2. 构建模型;
3. 模型训练;
4. 调优,反复训练;
5. 应用;
